<a href="https://colab.research.google.com/github/aswit3/Tamil_Number_MNIST_Research/blob/master/Tamil_Number_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#!unzip drive/My\ Drive/datasets/TAMIL_MNIST_DATASET/v1/TAMIL-MNIST-DATASET.zip -d drive/My\ Drive/datasets/TAMIL_MNIST_DATASET/v1

In [0]:
# import os
# import glob
# import csv

# from os import listdir
# from os.path import isfile, join

# Dir = "drive/My Drive/datasets/TAMIL_MNIST_DATASET/v1/TAMIL-MNIST-DATASET/"
# folders = [f for f in glob.glob(Dir + "**/", recursive=True)]

# for fn in folders[1:]:
#   j = (fn[len(fn)-2:]).split("/")[0]
#   changeDir = Dir+j+"/"

#   os.rename(fn, Dir+j)

#   fileList = [f for f in listdir(changeDir) if isfile(join(changeDir, f))]
#   for i, fname in enumerate(fileList):
#     i += 1
#     os.rename(changeDir+fname, changeDir+"{}_{}.jpg".format(j,i))
#   print("done")

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
from os import listdir
from os.path import isfile, join
import glob

import cv2
from tqdm import tqdm

In [0]:
training_data = []
IMG_SIZE = 100

for i in range(1,10):

  ImgDir = "drive/My Drive/datasets/TAMIL_MNIST_DATASET/v1/TAMIL-MNIST-DATASET/{}/".format(i)

  fileList = [ImgDir+f for f in listdir(ImgDir) if isfile(join(ImgDir, f))]

  for path1 in fileList:
    if os.path.isfile(path1):
        img1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)
        new_array = cv2.resize(img1, (IMG_SIZE, IMG_SIZE))
        training_data.append([new_array, i])
        # plt.imshow(new_array, cmap='gray')
        # plt.show()     
    else:
        print ("The file " + path1 + " does not exist.")

In [0]:
print(len(training_data))

In [0]:
import random
random.shuffle(training_data)

In [0]:
for sample in training_data[:10]:
  print(sample[1])

In [0]:
X = []
y = []

for features,label in training_data:
  X.append(features)
  y.append(label)

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [0]:
X.shape

In [0]:
import pickle
Dir = "drive/My Drive/datasets/TAMIL_MNIST_DATASET/v1/TAMIL-MNIST-DATASET/"

pickle_out = open(Dir+"X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open(Dir+"y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [0]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers

In [0]:
pickle_in = open(Dir+"X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open(Dir+"y.pickle","rb")
y = pickle.load(pickle_in)

In [0]:
X = X/255.0

model = Sequential()

model.add(Conv2D(256, (4, 4), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(9))
model.add(Activation('softmax'))

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(X, y, batch_size=1, epochs=10, validation_split=0.1)